In [15]:
from ucimlrepo import fetch_ucirepo

adult = fetch_ucirepo(id=2)

In [16]:
df = adult['data']['original']
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
48838,64,NaN,321403,HS-grad,9,Widowed,NaN,Other-relative,Black,Male,0,0,40,United-States,<=50K.
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


In [17]:
from dataclasses import dataclass
from typing import List, Tuple, Dict, Union
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, gaussian_kde
from sklearn.preprocessing import LabelEncoder


def is_numeric_column(series: pd.Series) -> bool:
    """
    Check if a column should be treated as numeric for KDE.
    """
    if not np.issubdtype(series.dtype, np.number):
        return False

    n_unique = len(series.dropna().unique())
    return n_unique >= 5


def is_integer_column(series: pd.Series) -> bool:
    """
    Check if a numeric column contains only integers.
    """
    return np.all(series.dropna() == series.dropna().astype(int))


def get_unique_samples(kde: gaussian_kde, n_samples: int, is_integer: bool = False,
                       max_attempts: int = 1000) -> np.ndarray:
    """
    Get unique samples from KDE with appropriate type handling.
    """
    samples = kde.resample(min(n_samples * 2, max_attempts))[0]

    if is_integer:
        samples = np.round(samples).astype(int)

    unique_samples = np.unique(samples)

    attempts = 1
    while len(unique_samples) < n_samples and attempts < max_attempts:
        new_samples = kde.resample(n_samples)[0]
        if is_integer:
            new_samples = np.round(new_samples).astype(int)
        unique_samples = np.unique(np.concatenate([unique_samples, new_samples]))
        attempts += 1

    if len(unique_samples) > n_samples:
        indices = np.linspace(0, len(unique_samples) - 1, n_samples).astype(int)
        unique_samples = np.sort(unique_samples)[indices]

    return unique_samples


@dataclass
class DataSchema:
    attr_categories: List[List[Union[int, float]]]  # Now starts from 0
    protected_attr: List[str]
    attr_names: List[str]
    binning_info: Dict[str, Dict[str, Union[List[float], str]]]
    kde_distributions: Dict[str, gaussian_kde]
    label_encoders: Dict[str, LabelEncoder]
    category_maps: Dict[str, Dict[Union[int, float], str]]


def create_kde_encoding(series: pd.Series, n_samples: int = 100) -> Tuple[
    np.ndarray, gaussian_kde, List[Union[int, float]], Dict[Union[int, float], str]]:
    """
    Create KDE from the series and sample fixed points from it.
    """
    non_nan_mask = pd.notna(series)
    values = series[non_nan_mask].to_numpy()

    if len(values) == 0:
        return np.full(len(series), -1), None, [-1, 0], {-1: 'nan', 0: '0'}

    kde = gaussian_kde(values)
    is_integer = is_integer_column(series)
    sampled_points = get_unique_samples(kde, n_samples, is_integer)

    # Create categories starting from -1 (missing) then 0 to n-1
    categories = [-1] + list(range(len(sampled_points)))

    # Create mapping dictionary
    if is_integer:
        category_map = {-1: 'nan',
                        **{i: str(int(point)) for i, point in zip(range(len(sampled_points)), sampled_points)}}
    else:
        category_map = {-1: 'nan',
                        **{i: f"{point:.3f}" for i, point in zip(range(len(sampled_points)), sampled_points)}}

    # Encode original values
    encoded = np.full(len(series), -1)  # Default to -1 for missing values
    for i, val in enumerate(series[non_nan_mask]):
        nearest_idx = np.abs(sampled_points - val).argmin()
        encoded[non_nan_mask][i] = nearest_idx  # Now starts from 0

    return encoded, kde, categories, category_map


def generate_schema_from_dataframe(
        df: pd.DataFrame,
        protected_columns: List[str] = None,
        attr_prefix: str = None,
        outcome_column: str = 'outcome',
        ensure_positive_definite: bool = True,
        n_samples: int = 100
) -> Tuple[DataSchema, np.ndarray]:
    """
    Generate a DataSchema and correlation matrix from a pandas DataFrame using KDE for numeric columns.
    """
    if outcome_column not in df.columns:
        raise ValueError(f"Outcome column '{outcome_column}' not found in DataFrame")

    if attr_prefix:
        attr_columns = [col for col in df.columns if col.startswith(attr_prefix)]
    else:
        attr_columns = [col for col in df.columns if col != outcome_column]

    if not attr_columns:
        raise ValueError("No attribute columns found")

    attr_categories = []
    encoded_df = pd.DataFrame(index=df.index)
    binning_info = {}
    kde_distributions = {}
    label_encoders = {}
    category_maps = {}

    for col in attr_columns:
        if is_numeric_column(df[col]):
            encoded_vals, kde, categories, category_map = create_kde_encoding(df[col], n_samples)

            encoded_df[col] = encoded_vals
            attr_categories.append(categories)  # Now contains [-1, 0, 1, ..., n-1]
            category_maps[col] = category_map

            if kde is not None:
                kde_distributions[col] = kde
                binning_info[col] = {
                    'strategy': 'kde',
                    'n_samples': n_samples,
                    'is_integer': is_integer_column(df[col])
                }

        else:
            le = LabelEncoder()
            non_nan_vals = df[col].dropna().unique()

            if len(non_nan_vals) > 0:
                str_vals = [str(x) for x in non_nan_vals]
                str_vals = list(dict.fromkeys(str_vals))

                le.fit(str_vals)
                encoded = np.full(len(df), -1)  # Default to -1 for missing values

                mask = df[col].notna()
                if mask.any():
                    encoded[mask] = le.transform([str(x) for x in df[col][mask]])  # Now starts from 0

                # Store encoded categories and mapping
                categories = [-1] + list(range(len(str_vals)))  # [-1, 0, 1, ..., n-1]
                category_map = {-1: 'nan', **{i: val for i, val in enumerate(str_vals)}}

                label_encoders[col] = le
                category_maps[col] = category_map
            else:
                encoded = np.full(len(df), -1)
                categories = [-1, 0]  # Always include 0 even if empty
                category_map = {-1: 'nan', 0: 'empty'}

            encoded_df[col] = encoded
            attr_categories.append(categories)

    # Rest of the function remains the same...
    if protected_columns is None:
        protected_attr = [False] * len(attr_columns)
    else:
        invalid_cols = [col for col in protected_columns if col not in attr_columns]
        if invalid_cols:
            raise ValueError(f"Protected columns {invalid_cols} not found in attributes")
        protected_attr = [col in protected_columns for col in attr_columns]

    correlation_matrix = np.zeros((len(attr_columns), len(attr_columns)))
    for i, col1 in enumerate(attr_columns):
        for j, col2 in enumerate(attr_columns):
            if i == j:
                correlation_matrix[i, j] = 1.0
            else:
                # Update mask to check for non-zero values
                mask = (encoded_df[col1] > 0) & (encoded_df[col2] > 0)
                if mask.any():
                    corr, _ = spearmanr(encoded_df[col1][mask], encoded_df[col2][mask])
                    correlation_matrix[i, j] = corr if not np.isnan(corr) else 0.0
                    correlation_matrix[j, i] = correlation_matrix[i, j]
                else:
                    correlation_matrix[i, j] = 0.0
                    correlation_matrix[j, i] = 0.0

    if ensure_positive_definite:
        eigenvalues, eigenvectors = np.linalg.eigh(correlation_matrix)
        if np.any(eigenvalues < 0):
            eigenvalues[eigenvalues < 0] = 1e-6
            correlation_matrix = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T
            scaling = np.sqrt(np.diag(correlation_matrix))
            correlation_matrix = correlation_matrix / scaling[:, None] / scaling[None, :]

    schema = DataSchema(
        attr_categories=attr_categories,
        protected_attr=protected_attr,
        attr_names=attr_columns,
        binning_info=binning_info,
        kde_distributions=kde_distributions,
        label_encoders=label_encoders,
        category_maps=category_maps
    )

    return schema, correlation_matrix


def decode_dataframe(df: pd.DataFrame, schema: DataSchema) -> pd.DataFrame:
    """
    Decode a dataframe using the schema's category maps.
    """
    decoded_df = pd.DataFrame(index=df.index)

    for col in schema.attr_names:
        if col in df.columns:
            category_map = schema.category_maps[col]
            decoded_df[col] = df[col].map(lambda x: category_map.get(x, category_map[0]))

    return decoded_df

In [18]:
schema, corr_matrix = generate_schema_from_dataframe(df, protected_columns=['race', 'sex'], outcome_column='income',
                                                     n_samples=10)

In [21]:
schema

DataSchema(attr_categories=[[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8], [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [-1, 0, 1, 2, 3, 4, 5, 6], [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], [-1, 0, 1, 2, 3, 4, 5], [-1, 0, 1, 2, 3, 4], [-1, 0, 1], [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]], protected_attr=[False, False, False, False, False, False, False, False, True, True, False, False, False, False], attr_names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'], binning_info={'age': {'strategy': 'kde', 

In [20]:
corr_matrix

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         3.23821747e-03,  0.00000000e+00, -8.65481335e-02,
         1.31005186e-02, -2.26928347e-02,  6.28912563e-02,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.87420798e-03],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  3.23821747e-03,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  2.01870765e-02,
        -3.75523035e-02,  1.36577585e-02,  1.

In [19]:
from data_generator.main import generate_data
from sklearn.preprocessing import LabelEncoder

data = generate_data(
    correlation_matrix=corr_matrix,
    data_schema=schema,
    prop_protected_attr=0.4,
    nb_groups=10,
    max_group_size=400,
    categorical_outcome=True,
    use_cache=False,
    corr_matrix_randomness=0.0)

print(f"Generated {len(data.dataframe)} samples in {data.nb_groups} groups")
print(f"Collisions: {data.collisions}")

Generating data:   0%|          | 0/10 [00:00<?, ?it/s]


ValueError: probabilities contain NaN

In [ ]:
import numpy as np
from scipy import stats
from scipy.spatial.distance import jensenshannon
import pandas as pd


def compare_datasets(df1, df2):
    results = {}

    for column in df1.columns:
        if np.issubdtype(df1[column].dtype, np.number):
            # KS test
            ks_stat, ks_pval = stats.ks_2samp(df1[column], df2[column])

            # Summary statistics
            mean_diff = abs(df1[column].mean() - df2[column].mean())
            std_ratio = df1[column].std() / df2[column].std()

            # Distribution comparison
            hist1, bins = np.histogram(df1[column], bins='auto', density=True)
            hist2, _ = np.histogram(df2[column], bins=bins, density=True)
            js_div = jensenshannon(hist1 + 1e-10, hist2 + 1e-10)

            results[column] = {
                'ks_statistic': ks_stat,
                'ks_pvalue': ks_pval,
                'mean_difference': mean_diff,
                'std_ratio': std_ratio,
                'jensen_shannon_div': js_div
            }

    # Correlation structure comparison
    corr1 = df1.corr()
    corr2 = df2.corr()
    corr_diff = np.abs(corr1 - corr2).mean().mean()
    results['correlation_matrix_difference'] = corr_diff

    return results